In [ ]:
# === 2) VERİ ÖNİŞLEME ===
import tensorflow as tf, matplotlib.pyplot as plt, numpy as np
from tensorflow.keras import layers

IMG, BATCH, SEED = 128, 32, 42
TRAIN_DIR = "/kaggle/input/intel-image-classification/seg_train/seg_train"
TEST_DIR  = "/kaggle/input/intel-image-classification/seg_test/seg_test"

# train/val ayrımı (validation_split)
train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR, validation_split=0.2, subset="training", seed=SEED,
    image_size=(IMG, IMG), batch_size=BATCH)

val_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR, validation_split=0.2, subset="validation", seed=SEED,
    image_size=(IMG, IMG), batch_size=BATCH)

test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR, image_size=(IMG, IMG), batch_size=BATCH)

class_names = train_ds.class_names
num_classes = len(class_names)

# Normalizasyon
norm = layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x,y: (norm(x), y))
val_ds   = val_ds.map(lambda x,y: (norm(x), y))
test_ds  = test_ds.map(lambda x,y: (norm(x), y))

# Performans
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(AUTOTUNE)
val_ds   = val_ds.cache().prefetch(AUTOTUNE)
test_ds  = test_ds.cache().prefetch(AUTOTUNE)

# Data Augmentation (Rotation, Flip, Zoom, Color Jitter benzeri: Contrast)
aug = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1)
], name="augmentation")

# Hızlı görselleştirme (istatistik/gözlem)
batch_x, batch_y = next(iter(train_ds))
plt.figure(figsize=(7,7))
for i in range(9):
    ax = plt.subplot(3,3,i+1)
    plt.imshow(aug(batch_x)[i].numpy())
    plt.title(class_names[int(batch_y[i].numpy())])
    plt.axis("off")
plt.show()


In [ ]:
# === 3) MODELİN EĞİTİLMESİ (CNN) ===
from tensorflow.keras import models

model = models.Sequential([
    layers.Input((IMG,IMG,3)),
    aug,  # augmentation

    layers.Conv2D(32, (3,3), padding="same", activation="relu"),  # Convolution + ReLU
    layers.MaxPooling2D(2,2),                                     # Pooling

    layers.Conv2D(64, (3,3), padding="same", activation="relu"),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128,(3,3), padding="same", activation="relu"),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(128, activation="relu"),                         # Dense + ReLU
    layers.Dropout(0.5),                                          # Dropout
    layers.Dense(num_classes, activation="softmax")               # Dense + Softmax (çıktı)
])

model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

cb = [
    tf.keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5)
]

EPOCHS = 15
hist = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=cb)


In [ ]:
# === 4A) Accuracy / Loss grafikleri ===
plt.figure(); plt.plot(hist.history["accuracy"]); plt.plot(hist.history["val_accuracy"])
plt.title("Accuracy"); plt.legend(["train","val"]); plt.show()
plt.figure(); plt.plot(hist.history["loss"]); plt.plot(hist.history["val_loss"])
plt.title("Loss"); plt.legend(["train","val"]); plt.show()


In [ ]:
# === 4B) Confusion Matrix & Classification Report (test set) ===
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# test tahmini
y_true, y_pred = [], []
for x,y in test_ds:
    p = model.predict(x, verbose=0)
    y_true.extend(y.numpy())
    y_pred.extend(np.argmax(p, axis=1))
y_true, y_pred = np.array(y_true), np.array(y_pred)

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_names, yticklabels=class_names)
plt.ylabel("True"); plt.xlabel("Pred"); plt.tight_layout(); plt.show()

print(classification_report(y_true, y_pred, target_names=class_names))


In [19]:
model.summary()
# === 4C) Grad-CAM — sağlam/bağımsız çözüm ===
import tensorflow as tf, numpy as np, matplotlib.pyplot as plt, cv2
from tensorflow.keras import layers, backend as K

# 0) Mevcut modeli fonksiyonel sarmalayıcıya dönüştür (model çağrılmış olur)
inp = tf.keras.Input(shape=(IMG, IMG, 3))
out = model(inp, training=False)       # <-- burada model bir kez kesin olarak 'call' edilir
wrapper_model = tf.keras.Model(inputs=inp, outputs=out)

# 1) Son konv katman adını güvenli seç
last_conv_name = None
for lyr in reversed(wrapper_model.layers):
    if isinstance(lyr, (layers.Conv2D, layers.SeparableConv2D, layers.DepthwiseConv2D)):
        last_conv_name = lyr.name
        break
assert last_conv_name is not None, "Conv katmanı bulunamadı."

# 2) Ara-çıktı modeli (hedef konv + nihai çıktı)
target_layer = wrapper_model.get_layer(last_conv_name)
cam_model = tf.keras.Model(
    inputs  = wrapper_model.inputs,
    outputs = [target_layer.output, wrapper_model.output]
)

# 3) Testten tek örnek al
sample_batch = next(iter(test_ds))
img = sample_batch[0][0:1]  # (1,H,W,3)

# 4) Grad-CAM hesabı
with tf.GradientTape() as tape:
    conv_out, preds = cam_model(img, training=False)
    cls  = tf.argmax(preds[0])
    loss = preds[:, cls]

grads = tape.gradient(loss, conv_out)
w = tf.reduce_mean(grads, axis=(0,1,2))
cam = tf.reduce_sum(conv_out[0] * w, axis=-1)
cam = tf.maximum(cam, 0) / (tf.reduce_max(cam) + 1e-8)
cam = tf.image.resize(cam[..., None], (IMG, IMG)).numpy()[..., 0]

# 5) Bindirme görseli
base_img = (img[0].numpy() * 255).astype("uint8")
heat     = (255 * cam).astype("uint8")
overlay  = cv2.addWeighted(base_img, 0.6,
                           cv2.applyColorMap(heat, cv2.COLORMAP_JET), 0.4, 0)

plt.figure(figsize=(7,3))
plt.subplot(1,2,1); plt.imshow(base_img); plt.title(f"Pred: {class_names[int(cls)]}"); plt.axis("off")
plt.subplot(1,2,2); plt.imshow(overlay);  plt.title("Grad-CAM"); plt.axis("off")
plt.tight_layout(); plt.show()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ augmentation (Sequential)       │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     4,194,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,865,364 (49.08 MB)

 Trainable params: 4,288,454 (16.36 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 8,576,910 (32.72 MB)

AssertionError: Conv katmanı bulunamadı.

In [18]:
# === 5) HİPERPARAMETRE OPTİMİZASYONU (manuel arama örneği) ===
from tensorflow.keras import models

def build_model(filters=(32,64,128), kernel=(3,3),
                dense_units=128, dropout=0.5,
                lr=1e-3, optimizer="adam"):
    m = models.Sequential([
        layers.Input((IMG,IMG,3)), aug,
        layers.Conv2D(filters[0], kernel, padding="same", activation="relu"),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(filters[1], kernel, padding="same", activation="relu"),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(filters[2], kernel, padding="same", activation="relu"),
        layers.MaxPooling2D(2,2),
        layers.Flatten(),
        layers.Dense(dense_units, activation="relu"),
        layers.Dropout(dropout),
        layers.Dense(num_classes, activation="softmax")
    ])
    if optimizer=="adam":
        opt = tf.keras.optimizers.Adam(lr)
    elif optimizer=="sgd":
        opt = tf.keras.optimizers.SGD(lr, momentum=0.9)
    elif optimizer=="rmsprop":
        opt = tf.keras.optimizers.RMSprop(lr)
    m.compile(optimizer=opt, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return m

search = [
    {"filters":(32,64,128), "kernel":(3,3), "dense_units":128, "dropout":0.5, "lr":1e-3,  "optimizer":"adam",  "batch":32},
    {"filters":(32,64,128), "kernel":(5,5), "dense_units":256, "dropout":0.5, "lr":5e-4, "optimizer":"adam",  "batch":64},
    {"filters":(64,128,256),"kernel":(3,3), "dense_units":256, "dropout":0.4, "lr":1e-3,  "optimizer":"sgd",   "batch":32},
]

results = []
for cfg in search:
    m = build_model(filters=cfg["filters"], kernel=cfg["kernel"],
                    dense_units=cfg["dense_units"], dropout=cfg["dropout"],
                    lr=cfg["lr"], optimizer=cfg["optimizer"])
    h = m.fit(train_ds, validation_data=val_ds, epochs=6, batch_size=cfg["batch"],
              callbacks=[tf.keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)], verbose=0)
    best_val = max(h.history["val_accuracy"])
    results.append((cfg, best_val))

results  # (denenen konfigürasyon, en iyi val_accuracy)


[({'filters': (32, 64, 128),
   'kernel': (3, 3),
   'dense_units': 128,
   'dropout': 0.5,
   'lr': 0.001,
   'optimizer': 'adam',
   'batch': 32},
  0.782252311706543),
 ({'filters': (32, 64, 128),
   'kernel': (5, 5),
   'dense_units': 256,
   'dropout': 0.5,
   'lr': 0.0005,
   'optimizer': 'adam',
   'batch': 64},
  0.7943692207336426),
 ({'filters': (64, 128, 256),
   'kernel': (3, 3),
   'dense_units': 256,
   'dropout': 0.4,
   'lr': 0.001,
   'optimizer': 'sgd',
   'batch': 32},
  0.6732003092765808)]